# Implementing an untyped Lambda Calculus


To introduce `metadsl` we will show how to create a working version of the untyped lambda calculus.

This will let us create the some basic combinators, verify that they before as expected under equiavelency.

Then we can also create an implementation of arithmetic on natural numbers, using church encoding and show how to convert
between these and Python's integers.


## Terms
The lambda calculus might sound abstract and odd, but if you know how to write functions in Python you already have mastered it!
The idea is a very simple language that only has functions that take in one argument.
Nothing, else, which seems a bit silly maybe, and it is not very useful by itself, but when it is integrated into a system with other data
types, it becomes a useful so that we can represent functions.


First, let's create a concept of a "variable" that can optionally have a name. It should only be equal to itself, so that we don't have to worry about shadowing variable names:

In [1]:
import dataclasses
import typing


@dataclasses.dataclass(eq=False)
class Variable:
    name: typing.Optional[str] = None
    
    def __repr__(self):
        return self.name or f"@{id(self)}"
        
x1 = Variable("x")
x2 = Variable("x")

assert x1 == x1
assert x1 != x2
assert hash(x1) != hash(x2)

Now, let's build our one and only key abstraction, a lambda term.
You should be able to call it and also create it by having a body and a variable. 

In [2]:
import metadsl


class LambdaTerm(metadsl.Instance):
    @metadsl.call(lambda self, arg: metadsl.instance_type(LambdaTerm))
    def __call__(self, arg: "LambdaTerm") -> "LambdaTerm":
        ...

@metadsl.call(lambda variable, body: metadsl.instance_type(LambdaTerm))
def abstraction(variable: Variable, body: LambdaTerm) -> LambdaTerm:
    ...

def create_abstraction(fn: typing.Callable[[LambdaTerm], LambdaTerm]) -> LambdaTerm:
    arg = Variable()
    return abstraction(arg, fn(LambdaTerm(arg)))

We can construct the identity function

In [3]:
@create_abstraction
def identity(x):
    return x

In [4]:
identity

LambdaTerm(abstraction(@4531825072, LambdaTerm(@4531825072)))

and then call it on a term:

In [5]:
y_term = LambdaTerm(Variable("y"))
called_identity = identity(y_term)
called_identity

LambdaTerm(__call__(LambdaTerm(abstraction(@4531825072, LambdaTerm(@4531825072))), LambdaTerm(y)))

This is a bit of a verbose way to print these.

Let's create a custom function that walks the tree and converts them to a string:

In [6]:
def pp_variable(variable: object):
    return (
        LambdaTerm(variable),
        lambda: LambdaTerm(str(variable)) if isinstance(variable, Variable) else None
    )

def pp_abstraction(variable: Variable, body: object):
    return (
        abstraction(variable, LambdaTerm(body)),
        lambda: LambdaTerm(f"(λ{variable}.{body})") if isinstance(body, str) else None,
    )

def pp_application(fn: object, arg: object):
    return (
        LambdaTerm(fn)(LambdaTerm(arg)),
        lambda: LambdaTerm(f"({fn} {arg})") if isinstance(fn, str) and isinstance(arg, str) else None,
    )

pretty_print = metadsl.replace_all(pp_variable, pp_abstraction, pp_application)

In [7]:
called_identity

LambdaTerm(__call__(LambdaTerm(abstraction(@4531825072, LambdaTerm(@4531825072))), LambdaTerm(y)))

In [8]:
metadsl.Expression.from_instance(called_identity)

Expression(type=LambdaTerm, value=__call__(Expression(type=LambdaTerm, value=abstraction(@4531825072, Expression(type=LambdaTerm, value=@4531825072))), Expression(type=LambdaTerm, value=y)))

In [9]:
pretty_print(called_identity)

LambdaTerm('((λ@4531825072.@4531825072) y)')

To make this useful, we need to define some semantics on these.

What's a useful thing to do with the lambda calculus?

Well it would be nice to actually be able to "compute" these values by replacing all instances of a variable with it's argument.

So let's replace all instances of calling a variable by reaplacing the arg inside of it:

In [10]:
@metadsl.replace_all
def beta_reduce(var: Variable, body: LambdaTerm, arg: LambdaTerm):
    # replaces all instances of var with arg 
    replace_var = metadsl.replace_all(lambda: (LambdaTerm(var), (lambda: arg)))
    
    return (
        abstraction(var, body)(arg),
        lambda: replace_var(body)
    )

Now let's combine these two, to execute them in sequence. First we want to beta reduce, until there is no more to beta reduce, then
we wanna turn that into a pretty print representation:

In [11]:
def execute(expr: LambdaTerm) -> str:
    return pretty_print(beta_reduce(expr)).__value__

In [12]:
print(execute(called_identity))

y
